Datasets:
- Snopes (D1)
- News dataset (D2)
- Fake and Real news (D3)

In [15]:
import pandas as pd
import numpy as np

path = "data/"
%ls

config_cpu.sh                data_sampled.csv   slurm-3274223.out
config_gpu.sh                LICENSE            slurm-3274757.out
data/                        models/            slurm-3275253.out
data_cleaning.ipynb          NB.ipynb           slurm-3304706.out
data_cleaning_snopes.ipynb   __pycache__/       translation/
data_download.ipynb          README.md          translation_eval.ipynb
data_gen_transformers.ipynb  slurm-3134754.out  tra.py


# News Datasest (News)

In [53]:
import re

d2_true = pd.read_csv(path + "News/" + "True.csv")
print(d2_true.columns)

d2_true["label"] = "true"
d2_true = d2_true[["text", "label"]]

d2_false = pd.read_csv(path + "News/" + "Fake.csv")
d2_false["label"] = "false"
d2_false = d2_false[["text", "label"]]

print(d2_true.shape, d2_false.shape)
d2 = pd.concat([d2_true, d2_false])

d2["text"] = [" ".join(text.split(" - ")[1:]) if len(text.split(" - ")) > 1 else text for text in d2["text"]]
d2["text"] = [re.sub(r"((https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|bit.ly/[\w]+)", " ", text) for text in d2["text"]]
d2["text"] = [re.sub(r"(?<![A-Z])([\W]+\B)", " ", text) for text in d2["text"]]
d2["text"] = [re.sub(r"\[.+\]", " ", text) for text in d2["text"]]
d2["text"] = [re.sub(r" {2,}", " ", text) for text in d2["text"]]

d2["text"], d2["label"] = d2["label"], d2["text"]
d2.columns = ["label", "claim"]

d2 = d2.drop_duplicates("claim")

print(d2.shape)
print(d2.columns)
# print(d2)
print(d2.value_counts("label"))
d2.to_csv(path + "News/" + "news_cleaned.csv", index=False, sep="|")

Index(['title', 'text', 'subject', 'date'], dtype='object')
(21417, 2) (23481, 2)
(38588, 2)
Index(['label', 'claim'], dtype='object')
label
true     21182
false    17406
dtype: int64


# FR news (fake_and_real_news_dataset.csv)

In [91]:
import math

d_fr = pd.read_csv("data/FR_News/fake_and_real_news_dataset.csv")
d_fr = d_fr[[ "label", "text"]]
d_fr.columns = ["label", "claim"]

d_fr["label"] = d_fr["label"].replace(["REAL", "FAKE"], ["true", "false"])

d_fr["claim"] = [" ".join(text.split(r" -+ ")[1:]) if len(text.split(" - ")) > 1 else text for text in d_fr["claim"]]

d_fr["claim"] = [re.sub(r"((https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|bit.ly/[\w]+)", " ", text) for text in d_fr["claim"]]
d_fr["claim"] = [re.sub(r"(?<![A-Z])([\W]+\B)", " ", text) for text in d_fr["claim"]]
d_fr["claim"] = [re.sub(r"\[.+\]", " ", text) for text in d_fr["claim"]]

d_fr["claim"] = [re.sub(r"[\"\“\`]+", " ", text) for text in d_fr["claim"]]
d_fr["claim"] = [re.sub(r"\s+", " ", text) for text in d_fr["claim"]]
d_fr["claim"] = [re.sub(r"\s{2,}", " ", text) for text in d_fr["claim"]]
d_fr["claim"] = [math.nan if len(text) < 1 else text for text in d_fr["claim"]]

d_fr = d_fr.drop_duplicates("claim")
d_fr = d_fr.dropna()

print(d_fr)
d_fr.to_csv(path + "FR_News/" + "fr_news_cleaned.csv", index=False, sep="|")

      label                                              claim
0      true  UPDATE: Gov Fallin vetoed the bill on Friday H...
1      true  Ever since Texas laws closed about half of the...
2      true  Donald Trump and Hillary Clinton now at the st...
3      true  A Houston grand jury investigating criminal al...
4      true  WASHINGTON Forty-three years after the Supreme...
...     ...                                                ...
4588  false  Support Us Should I Get Botox source Add To Th...
4590  false  Print Sen Bernie Sanders laid out the ways he ...
4591  false  As the pressure to start construction on the D...
4592  false  Tweet Home Gold Gold News Currency Crisis Alas...
4593  false  Written by Eric Margolis As a former soldier a...

[4089 rows x 2 columns]


In [61]:
t = d_fr["claim"].to_numpy()

for text in t[-100:]:
    print(text)
    print("-----")

Leave a reply Steve Taylor Ph.D From the standpoint of ordinary consciousness separateness seems to be a basic part of the human condition Most human beings experience themselves as egos trapped inside their own mind-space observing a world which seems to be out there on the other side of their skulls As a result the normal human state is one of aloneness We’re always onlookers rather than participants We can communicate with other people by speaking writing or gesturing but they will never be able to truly know us or to share our thoughts and feelings Our inner being will always be sealed off from them Ego-separation also creates a sense of incompleteness Because we’re separate from the world we’re like fragments which have broken off from the whole and so feel a sense of insufficiency There s a kind of hole inside us which we spend most of our lives trying to fill (but very rarely manage to like cats who were taken away from their mother at birth and who are always hankering for affe

# Combined dataset + statistics

In [23]:
d_snopes = pd.read_csv(path + "Snopes/" + "cleaned_snopes.csv", sep="|")
d_news = pd.read_csv(path + "News/" + "news_cleaned.csv", sep="|")
d_fr = pd.read_csv(path + "FR_News/" + "fr_news_cleaned.csv", sep="|")

print(d_snopes.head(), d_snopes.shape)
print(d_news.head(), d_news.shape)
print(d_fr.head(), d_fr.shape)

   label                                              claim
0  False  A man claiming to be transgender allegedly ass...
1  False  An aide handed Sean Spicer a note that read “Y...
2  False  Hip-hop and pop star Nelly was arrested on dru...
3  False  President Obama pardoned Chelsea Manning and 1...
4   True               No two snowflakes are exactly alike. (4433, 2)
   label                                              claim
0  False  The head of a conservative Republican faction ...
1  False  Transgender people will be allowed for the fir...
2  False  The special counsel investigation of links bet...
3  False  Trump campaign adviser George Papadopoulos tol...
4  False  President Donald Trump called on the U.S. Post... (21193, 2)
   label                                              claim
0   True  UPDATE: Gov. Fallin vetoed the bill on Friday....
1   True  Ever since Texas laws closed about half of the...
2   True  Donald Trump and Hillary Clinton, now at the s...
3   True  A Houston

In [24]:
df = pd.concat([d_snopes, d_news, d_fr])

print(df.shape)
print(df.value_counts("label"))

df.to_csv(path + "data.csv", index=False, sep="|")

(30035, 2)
label
False    27174
True      2830
dtype: int64


In [29]:
import random
import sys

true_claims = df[df["label"] == True]
print(true_claims.shape)

aux_true = []
for idx, label, claim in true_claims.itertuples():
	sys.stdout.write("\r" + f"{idx}/{len(true_claims)}")
    # jumble words
	aux_claim = claim
	if random.random() < 0.6:
		aux_claim = aux_claim.split(" ")
		idx = 0
		while idx < len(aux_claim)-1:
			if random.random() < 0.5:
				random_idx = random.randint(0, len(aux_claim)-1)
				aux_claim[idx], aux_claim[random_idx] = aux_claim[random_idx], aux_claim[idx]
			idx += 1
		aux_claim = " ".join(aux_claim)
	# add text multiple times
	if random.random() < 0.4:
		aux_true.append([True, aux_claim])
	aux_true.append([True, aux_claim])
	
print()
aux_true = pd.DataFrame(aux_true, columns=["label", "claim"]) 
aux_true = pd.concat([aux_true, true_claims])
aux_true = aux_true.reset_index()
aux_true = aux_true.drop("index", axis=1)
print(aux_true)
print(aux_true.shape)

(2830, 2)
2170/2830
     label                                              claim
0     True               No two snowflakes are exactly alike.
1     True  Morris Moe Berg, a multilingual Princeton Univ...
2     True  A video clip shows a Budweiser tribute to the ...
3     True  Actor Channing Tatum and dancer and actor Jenn...
4     True  Actor Channing Tatum and dancer and actor Jenn...
...    ...                                                ...
6813  True  Donald Trump on Monday spent part of his July ...
6814  True  NEW YORK — His campaign in turmoil, Donald Tru...
6815  True  Peggy Drexler is the author of "Our Fathers, O...
6816  True  Donald Trump returns to the campaign trail fro...
6817  True  A lot of people don’t think Donald Trump would...

[6818 rows x 2 columns]
(6818, 2)


/tmp/ipykernel_2033354/2085465268.py:28: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  aux_true = pd.concat([aux_true, true_claims])


In [31]:
false_claims = df[df["label"] == False]
df = pd.concat([false_claims, aux_true])
print(df.value_counts("label"))
print(df)
df.to_csv(path + "data_sampled.csv", index=None, sep="|")

label
False    27174
True      6818
dtype: int64
      label                                              claim
0     False  A man claiming to be transgender allegedly ass...
1     False  An aide handed Sean Spicer a note that read “Y...
2     False  Hip-hop and pop star Nelly was arrested on dru...
3     False  President Obama pardoned Chelsea Manning and 1...
6     False  Raggedy Ann dolls were created by a grief-stri...
...     ...                                                ...
6813   True  Donald Trump on Monday spent part of his July ...
6814   True  NEW YORK — His campaign in turmoil, Donald Tru...
6815   True  Peggy Drexler is the author of "Our Fathers, O...
6816   True  Donald Trump returns to the campaign trail fro...
6817   True  A lot of people don’t think Donald Trump would...

[33992 rows x 2 columns]
